we'll use our cleaned data to apply the provided tasks using SparkSQL

In [2]:
# Installing required packages
!pip install pyspark
!pip install findspark
!pip install pyarrow==1.0.0
!pip install pandas
!pip install numpy==1.19.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=5a5e082545f71bf5ec9f8beb535201178ff27163f5d36245af2961c3ce664026
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subproc

In [3]:
import findspark
findspark.init()

In [4]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [5]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [22]:
spark

In [38]:
# Read the file using `read_csv` function in pandas
data = pd.read_csv('/content/cleaned_music.csv')
data.head()

,Artist_Name,Track_Name,Popularity,danceability,energy,key,loudness_in_(dB),mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo_(BPM),Duration_inMins,Genre
0,Boston,Hitch a Ride,54.0,0.382,0.814,pitch D# or Eb,-7.230,Major,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,4.195550,Country
1,The Raincoats,No Side to Fall In,35.0,0.434,0.614,F# or Gb,-8.334,Major,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,1.827783,Alt_Music
2,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,pitch D,-4.279,Major,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,3.832667,Country
3,The Stooges,Search and Destroy - Iggy Pop Mix,53.0,0.235,0.977,F# or Gb,0.878,Major,0.1070,0.003530,0.006040,0.1720,0.2410,152.952,3.468883,Alt_Music
4,Solomon Burke,None Of Us Are Free,48.0,0.674,0.658,pitch F,-9.647,Minor,0.1040,0.404000,0.000001,0.0981,0.6770,143.292,5.489783,Alt


In [40]:
data['Track_Name'].count()

10338

In [24]:
spark_df2 = spark.createDataFrame(data) 

In [25]:
spark_df2.printSchema()

root
 |-- Artist_Name: string (nullable = true)
 |-- Track_Name: string (nullable = true)
 |-- Popularity: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness_in_(dB): double (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo_(BPM): double (nullable = true)
 |-- Duration_inMins: double (nullable = true)
 |-- Genre: string (nullable = true)



In [ ]:
spark_df2.createTempView("music") # already exist 

In [39]:
# Showing the whole table
spark.sql("SELECT * FROM music").show()

+--------------------+--------------------+----------+------------+------+--------------+----------------+-----+-----------+------------+----------------+--------+-------+-----------+------------------+-------------+
|         Artist_Name|          Track_Name|Popularity|danceability|energy|           key|loudness_in_(dB)| mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo_(BPM)|   Duration_inMins|        Genre|
+--------------------+--------------------+----------+------------+------+--------------+----------------+-----+-----------+------------+----------------+--------+-------+-----------+------------------+-------------+
|              Boston|        Hitch a Ride|      54.0|       0.382| 0.814|pitch D# or Eb|           -7.23|Major|     0.0406|      0.0011|         0.00401|   0.101|  0.569|    116.454|           4.19555|      Country|
|       The Raincoats|  No Side to Fall In|      35.0|       0.434| 0.614|      F# or Gb|          -8.334|Major|     0.0525|       0

**Part B:**  
**Remove any songs that exceed 5 minutes**

to do that, we'll select all the column from the music table where the condition : duration < = 5 min , and then, we'll save this query in the variable spark_df2. As shown below: 

In [28]:
spark_df2 = spark.sql("SELECT * FROM music WHERE Duration_inMins <= 5")
spark_df2.show()

+--------------------+--------------------+----------+------------+------+--------------+----------------+-----+-----------+------------+----------------+--------+-------+-----------+------------------+-------------+
|         Artist_Name|          Track_Name|Popularity|danceability|energy|           key|loudness_in_(dB)| mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo_(BPM)|   Duration_inMins|        Genre|
+--------------------+--------------------+----------+------------+------+--------------+----------------+-----+-----------+------------+----------------+--------+-------+-----------+------------------+-------------+
|              Boston|        Hitch a Ride|      54.0|       0.382| 0.814|pitch D# or Eb|           -7.23|Major|     0.0406|      0.0011|         0.00401|   0.101|  0.569|    116.454|           4.19555|      Country|
|       The Raincoats|  No Side to Fall In|      35.0|       0.434| 0.614|      F# or Gb|          -8.334|Major|     0.0525|       0

however, this change will not reflect on spark itself, to make this change on spark we'll apply the following :  

In [29]:
spark_df2.createOrReplaceTempView("songs")
# by using this function, we'll save the changes in a new tempview that called songs,
# and this is gonna be our updated table and we'll be using it to solve the rest of the questions

In [30]:
spark.sql("SELECT * FROM songs ").show()
# display the updated table (songs)

+--------------------+--------------------+----------+------------+------+--------------+----------------+-----+-----------+------------+----------------+--------+-------+-----------+------------------+-------------+
|         Artist_Name|          Track_Name|Popularity|danceability|energy|           key|loudness_in_(dB)| mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo_(BPM)|   Duration_inMins|        Genre|
+--------------------+--------------------+----------+------------+------+--------------+----------------+-----+-----------+------------+----------------+--------+-------+-----------+------------------+-------------+
|              Boston|        Hitch a Ride|      54.0|       0.382| 0.814|pitch D# or Eb|           -7.23|Major|     0.0406|      0.0011|         0.00401|   0.101|  0.569|    116.454|           4.19555|      Country|
|       The Raincoats|  No Side to Fall In|      35.0|       0.434| 0.614|      F# or Gb|          -8.334|Major|     0.0525|       0

**c) Display songs by J. Cole, Novo Amor and Anson Seabra** 

to display these songs, we'll apply the quesries separately to display a clear result as we tried to do it individually but it didn’t work, we'll simply select the artist name and track name and setting the condition by initializing the artist name to what's required in the question .

In [31]:
spark.sql("SELECT Artist_Name, Track_Name FROM songs WHERE Artist_Name = 'J. Cole'").show()

+-----------+----------+
|Artist_Name|Track_Name|
+-----------+----------+
+-----------+----------+



In [41]:
spark.sql("SELECT Artist_Name, Track_Name FROM songs WHERE Artist_Name = 'Novo Amor'").show(truncate=False)

+-----------+----------+
|Artist_Name|Track_Name|
+-----------+----------+
|Novo Amor  |Birdcage  |
+-----------+----------+



In [42]:
spark.sql("SELECT Artist_Name, Track_Name FROM songs WHERE Artist_Name = 'Anson Seabra'").show(truncate=False)

+------------+--------------------------+
|Artist_Name |Track_Name                |
+------------+--------------------------+
|Anson Seabra|It's Raining, It's Pouring|
+------------+--------------------------+



**d) How many songs are included in every category?**

to display the (count / total number) of songs in each genre/category, we'll use the count method the result will be displayed in a new column called( SongCount) to calculate the total number of songs in each genre where the condition will be grouping by the genre to display the final result. As shown below: 

In [34]:
spark.sql("SELECT Genre, COUNT(*) as SongCount FROM songs GROUP BY Genre").show()

+-------------+---------+
|        Genre|SongCount|
+-------------+---------+
|          Alt|      714|
| Instrumental|      800|
|    Alt_Music|     1548|
|        Metal|      137|
|         Rock|      297|
|        Blues|      275|
|Acoustic/Folk|      985|
|          Pop|      139|
|      Country|     2405|
|       HipHop|      373|
|        Indie|      851|
+-------------+---------+



**e) Which artists dominated the charts?**

same concept of spark dataframe will be applied here, first, we'll select the artist name. Then, we'll find the max popularity of each song and we will be grouping by the artist name with their corresponding max popularity but it will be sorted in a descending order. As shown below:

In [35]:
spark.sql("SELECT Artist_Name, MAX(Popularity) as MaxPopularity FROM songs GROUP BY Artist_Name ORDER BY MaxPopularity DESC").show()

+--------------------+-------------+
|         Artist_Name|MaxPopularity|
+--------------------+-------------+
|            Doja Cat|         98.0|
|          Ed Sheeran|         97.0|
|       Doja Cat, SZA|         95.0|
|      Olivia Rodrigo|         95.0|
|      Rauw Alejandro|         95.0|
|               Riton|         94.0|
|       Billie Eilish|         93.0|
|          Nio Garcia|         93.0|
|Riton, Nightcrawl...|         92.0|
|                 ATB|         91.0|
|        Bella Poarch|         90.0|
|          Joel Corry|         90.0|
|         Post Malone|         90.0|
|             Cardi B|         90.0|
|              AURORA|         89.0|
|            Lil Tjay|         89.0|
|  Majestic, Boney M.|         89.0|
|                 SZA|         89.0|
|            Saweetie|         89.0|
|            Majestic|         88.0|
+--------------------+-------------+
only showing top 20 rows



**f) What songs would be considered for the “Billboard Top 10 Songs of the Year”? (mention their artists as well)**

to find the top 10 songs, we'll select the track name, artist name, and popularity where we'll be grouping by the popularity in a desceding order with setting a limit to be = 10 as we just want the top 10 songs .

the reason of choosing the popularity to be our attribute that will sort the songs based on it is because these top songs are actually top because they're popular/ most trending . 

In [43]:
spark.sql("SELECT Artist_Name, Track_Name, Popularity FROM songs ORDER BY Popularity DESC LIMIT 10").show(truncate=False)


+--------------+--------------------------------------------------+----------+
|Artist_Name   |Track_Name                                        |Popularity|
+--------------+--------------------------------------------------+----------+
|Doja Cat      |Kiss Me More (feat. SZA)                          |98.0      |
|Ed Sheeran    |Bad Habits                                        |97.0      |
|Doja Cat, SZA |Kiss Me More (feat. SZA)                          |95.0      |
|Rauw Alejandro|Todo De Ti                                        |95.0      |
|Olivia Rodrigo|deja vu                                           |95.0      |
|Olivia Rodrigo|drivers license                                   |95.0      |
|Doja Cat, SZA |Kiss Me More (feat. SZA)                          |95.0      |
|Riton         |Friday (feat. Mufasa & Hypeman) - Dopamine Re-Edit|94.0      |
|Nio Garcia    |Am                                                |93.0      |
|Billie Eilish |Your Power                          

**g) Recommend at least 5 songs that can be played at a party**


we can apply the same query here as the same concept goes here with the only different fel limit to be = 5. Meaning that, to recommend songs in a party, we'll think of how the song is popular/ trending between people so they can really have fun :) 

( as the goal of a party is to dance and have fun with something we really like) 

In [44]:
spark.sql("SELECT Track_Name, Artist_Name, Popularity FROM songs ORDER BY Popularity DESC LIMIT 5").show(truncate=False)

+------------------------+--------------+----------+
|Track_Name              |Artist_Name   |Popularity|
+------------------------+--------------+----------+
|Kiss Me More (feat. SZA)|Doja Cat      |98.0      |
|Bad Habits              |Ed Sheeran    |97.0      |
|deja vu                 |Olivia Rodrigo|95.0      |
|drivers license         |Olivia Rodrigo|95.0      |
|Kiss Me More (feat. SZA)|Doja Cat, SZA |95.0      |
+------------------------+--------------+----------+

